In [20]:
%load_ext autoreload

In [23]:
%autoreload 1

In [25]:
%aimport src.preprocessing

In [27]:
from src.preprocessing import Cleaner

In [26]:
import difflib
from collections import Counter, Mapping
import math
from itertools import zip_longest
from Levenshtein import median, median_improve
import string
from pathlib import Path

In [29]:
cleaner = Cleaner(lowercase=True, table='punctuation')
cleaner('Aa.,;#fheekJK### \r\n karim')

'aa    fheekjk    \r\n karim'

In [2]:
def find_indices(list_, element):
    rez = []
    offset = -1
    while True:
        try:
            offset = list_.index(element, offset+1)
            rez.append(offset)
        except ValueError:
            return rez

In [3]:
def preprocess(data):
    return [el.lower().strip(" .\n\t") for el in data]

In [4]:
data = ['евтушенко', "евтушенко сергей", "Лозовая .", "Лозовая Мария", "лозовая",
         "сергей евтушенко", "Евтушенкоо", "Евтушенко Сергей", "мария лозовая",
        "United states", "United States", "Ukraine", "united states of americe", 
        "United States of America"]
print("Input: {0}\n".format(data))
data = preprocess(data)
temp_vocab = data
mapper = {}
while temp_vocab:
    word = temp_vocab[0]
    temp = difflib.get_close_matches(word, temp_vocab[1:], n = 2**64)
    group = [word, *temp]
    most_common = Counter(group).most_common(1)[0][0]
    mapper.update(list(zip_longest(set(temp), [word], fillvalue=word)))
    temp_vocab = [el for el in temp_vocab if el not in group]
    
for i,el in enumerate(data):
    data[i] = mapper.get(el, el)
print("Output: {0}".format(data))

Input: ['евтушенко', 'евтушенко сергей', 'Лозовая .', 'Лозовая Мария', 'лозовая', 'сергей евтушенко', 'Евтушенкоо', 'Евтушенко Сергей', 'мария лозовая', 'United states', 'United States', 'Ukraine', 'united states of americe', 'United States of America']

Output: ['евтушенко', 'евтушенко', 'лозовая', 'лозовая', 'лозовая', 'евтушенко', 'евтушенко', 'евтушенко', 'лозовая', 'united states', 'united states', 'ukraine', 'united states', 'united states']


In [6]:
def mode(group):
    return Counter(group).most_common(1)[0][0]

In [7]:
def median_string(group):
    return median(group)